# Microlocal Analysis, D-Modules & Categorical Resolution

**Theme: The Phase Space is the Reality**


## 1. Introduction: The Phase Space Resolution

In eps. 6, we treated differential equations as geometric surfaces in Jet Space. But we still encountered singularities where the "tangent planes" became vertical (caustics, shocks).

In eps. 7, we realize that singularities are not "failures" of the function, but rather geometric objects living in the **Phase Space** (the Cotangent Bundle $T^*M$). By lifting our perspective from position space ($M$) to phase space ($T^*M$), we can resolve these singularities.

We then algebrize this entire structure using **D-Modules**, turning differential equations into modules over a ring of operators. This culminates in the **Riemann-Hilbert Correspondence**, which reveals that the "solutions" to a differential equation are actually topological shapes (Sheaves) disguised as analysis.


## 2. Microlocal Analysis & The Wave Front Set

Classical smoothness ($C^\infty$) is a local property. Microlocal analysis refines this by asking *in which direction* a function is singular.

### 2.1 The Singular Support

A distribution $u$ is singular at $x$ if it cannot be multiplied by a smooth cutoff function $\phi$ to make its Fourier transform decay rapidly.

The **Wave Front Set** $WF(u) \subset T^*M$ collects the pairs $(x, \xi)$ where $u$ is non-smooth at $x$ in the direction $\xi$.

### 2.2 Propagation of Singularities

Hörmander (1971) proved that for a hyperbolic operator $P$ (like the Wave Operator), singularities don't just sit still; they travel along **Bi-characteristics** (Hamiltonian flow lines in phase space). This explains why sound "rays" exist: they are the tracks of singularities in the wave front set.



In [12]:
# [The Wave Front. A phase space visualization (x,p). The user defines a function with a corner (singularity). The plot highlights the "cone" of directions in phase space where the Fourier transform does not decay, visualizing the Wave Front Set.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class WaveFrontWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['sing_pos'] = ParameterSlider(
            name='sing_pos',
            value=0.0,
            min_val=-1.0,
            max_val=1.0,
            step=0.1,
            description='Singularity Position $x_0$'
        )
        self.sliders['sing_type'] = ParameterSlider(
            name='sing_type',
            value=0.0,
            min_val=0.0,
            max_val=4.0,
            step=1.0,
            description='Type (0: Jump, 1: Corner, 2: Delta, 3: Cusp, 4: Step)'
        )
        self.sliders['show_fourier'] = ParameterSlider(
            name='show_fourier',
            value=0.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Show Fourier Transform (0: Off, 1: On)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            sing_pos = params['sing_pos']
            stype = int(params['sing_type'])

            # Position space x
            x = np.linspace(-2, 2, 500)
            show_fourier = params['show_fourier'] > 0.5

            if stype == 0:
                # Jump discontinuity: Heaviside function
                f = np.heaviside(x - sing_pos, 1.0)
                title = 'Jump Discontinuity (Heaviside)'
                wf_description = "$p \\neq 0$"
            elif stype == 1:
                # Corner (cusp): absolute value
                f = np.abs(x - sing_pos)
                title = 'Corner (Cusp)'
                wf_description = "all $p$"
            elif stype == 2:
                # Delta function (approximated as narrow Gaussian)
                sigma = 0.05
                f = np.exp(-((x - sing_pos)**2) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
                title = 'Delta Function (Approx)'
                wf_description = "all $p$"
            elif stype == 3:
                # Cusp: |x|^α with α < 1
                alpha = 0.5
                f = np.abs(x - sing_pos)**alpha
                title = f'Cusp $|x|^{{\\alpha}}$'
                wf_description = "all $p$"
            else:  # stype == 4
                # Step with finite slope (smoothed jump)
                width = 0.1
                f = 0.5 * (1 + np.tanh((x - sing_pos) / width))
                title = 'Smooth Step (Tanh)'
                wf_description = "$p \\neq 0$ (weak)"

                        # Phase space (x, p) for wave front set
            # Wave front set WF(u) = {(x, ξ) ∈ T*M : u is not smooth at x in direction ξ}
            # For a jump discontinuity at x₀: WF = {(x₀, p) : p ≠ 0}
            #   This is a vertical line in phase space at x = x₀, for all non-zero momenta
            # For a corner (cusp) at x₀: WF = {(x₀, p) : p ∈ ℝ}
            #   This is a vertical line in phase space at x = x₀, for all momenta

            # Create phase space grid
            x_phase = np.linspace(-2, 2, 300)
            p = np.linspace(-10, 10, 300)
            X_phase, P_phase = np.meshgrid(x_phase, p)

            # Wave front set: vertical line at x = sing_pos
            # Initialize as zero everywhere
            wave_front = np.zeros_like(X_phase)

            # Find indices where x is close to singularity
            x_idx = np.argmin(np.abs(x_phase - sing_pos))
            x_sing = x_phase[x_idx]

            if stype in [0, 4]:  # Jump or smooth step
                # Jump discontinuity: WF = {(x₀, p) : p ≠ 0}
                # Exclude p = 0 (the zero momentum direction)
                p_mask = np.abs(P_phase) > 0.05  # Exclude p near zero
                wave_front[:, x_idx] = np.where(p_mask[:, x_idx], 1.0, 0.0)
                if stype == 4:
                    # Smooth step has weaker singularity, reduce intensity
                    wave_front[:, x_idx] *= 0.7
            else:  # Corner, Delta, Cusp
                # WF = {(x₀, p) : p ∈ ℝ} - all momenta are singular
                wave_front[:, x_idx] = 1.0

            # Create intensity visualization (decay away from singularity)
            # The wave front set is exactly at x = sing_pos, but we can show it with some width for visibility
            x_dist = np.abs(X_phase - sing_pos)
            intensity = np.where(x_dist < 0.05, wave_front, 0.0)  # Narrow band around singularity

            # Determine number of rows based on whether to show Fourier transform
            if show_fourier:
                rows = 3
                subplot_titles = (
                    'Function $f(x)$ with Singularity',
                    'Wave Front Set in Phase Space $(x, p)$',
                    'Fourier Transform $\\hat{f}(k)$ (Decay Analysis)'
                )
                row_heights = [0.3, 0.4, 0.3]
            else:
                rows = 2
                subplot_titles = (
                    'Function $f(x)$ with Singularity',
                    'Wave Front Set in Phase Space $(x, p)$'
                )
                row_heights = [0.4, 0.6]

            fig = make_subplots(
                rows=rows, cols=1,
                subplot_titles=subplot_titles,
                vertical_spacing=0.12,
                row_heights=row_heights
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=500,
                width=900,
                showlegend=True
            )

            # Top plot: Function f(x)
            fig.add_trace(
                go.Scatter(
                    x=x, y=f,
                    mode='lines',
                    name='$f(x)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=3)
                ),
                row=1, col=1
            )

            # Mark singularity position
            fig.add_trace(
                go.Scatter(
                    x=[sing_pos], y=[f[np.argmin(np.abs(x - sing_pos))]],
                    mode='markers',
                    marker=dict(size=12, color='red', symbol='x'),
                    name='Singularity'
                ),
                row=1, col=1
            )

            # Bottom plot: Wave Front Set in phase space (x, p)
            # The wave front set is a vertical line at x = sing_pos
            # Use heatmap to show the wave front set clearly
            # Normalize intensity for better visualization
            intensity_normalized = intensity / (np.max(intensity) + 1e-10)

            fig.add_trace(
                go.Heatmap(
                    x=x_phase,
                    y=p,
                    z=intensity_normalized,
                    colorscale=[[0, 'rgba(0,0,0,0)'], [0.3, 'rgba(50,100,200,0.4)'], [0.7, 'rgba(150,200,255,0.8)'], [1, 'rgba(255,150,100,1)']],
                    showscale=True,
                    colorbar=dict(title="Wave Front<br>Set Intensity", x=1.02),
                    name='Wave Front Set',
                    zmin=0,
                    zmax=1
                ),
                row=2, col=1
            )

            # Add explicit vertical line to highlight the wave front set
            # For jump: line at x = sing_pos, excluding p = 0
            # For corner: line at x = sing_pos, all p
            if stype == 0:
                # Jump: exclude p = 0 (exclude small p near zero)
                p_mask_line = np.abs(p) > 0.05
                p_line = p[p_mask_line]
                x_line = np.full_like(p_line, sing_pos)
            else:
                # Corner: all p (including p = 0)
                p_line = p
                x_line = np.full_like(p_line, sing_pos)

            # Add Fourier transform plot if requested
            if show_fourier:
                # Compute Fourier transform
                k = np.linspace(-50, 50, 1000)
                # Use FFT for efficiency
                dx = x[1] - x[0]
                f_fft = np.fft.fft(f)
                k_fft = np.fft.fftfreq(len(x), dx) * 2 * np.pi
                # Sort for plotting
                sort_idx = np.argsort(k_fft)
                k_fft_sorted = k_fft[sort_idx]
                f_fft_sorted = np.abs(f_fft[sort_idx])

                # Interpolate to k grid
                from scipy.interpolate import interp1d
                if len(k_fft_sorted) > 0:
                    interp_func = interp1d(k_fft_sorted, f_fft_sorted, kind='linear',
                                         bounds_error=False, fill_value=0.0)
                    f_k = interp_func(k)
                else:
                    f_k = np.zeros_like(k)

                # Plot Fourier transform
                fig.add_trace(
                    go.Scatter(
                        x=k, y=f_k,
                        mode='lines',
                        name='$|\\hat{f}(k)|$',
                        line=dict(color=self.plot_manager.config.colors[2], width=2)
                    ),
                    row=3, col=1
                )

                # Add decay reference lines
                if stype in [0, 4]:  # Jump: decays as 1/|k|
                    k_ref = k[np.abs(k) > 1]
                    decay_ref = 1.0 / np.abs(k_ref)
                    decay_ref = decay_ref / decay_ref[0] * f_k[np.abs(k) > 1][0] if len(decay_ref) > 0 else decay_ref
                    fig.add_trace(
                        go.Scatter(
                            x=k_ref, y=decay_ref,
                            mode='lines',
                            name='$\\sim 1/|k|$ (slow decay)',
                            line=dict(color='orange', width=2, dash='dash')
                        ),
                        row=3, col=1
                    )
                elif stype == 1:  # Corner: decays as 1/k²
                    k_ref = k[np.abs(k) > 1]
                    decay_ref = 1.0 / (k_ref**2)
                    decay_ref = decay_ref / decay_ref[0] * f_k[np.abs(k) > 1][0] if len(decay_ref) > 0 else decay_ref
                    fig.add_trace(
                        go.Scatter(
                            x=k_ref, y=decay_ref,
                            mode='lines',
                            name='$\\sim 1/k^2$ (faster decay)',
                            line=dict(color='orange', width=2, dash='dash')
                        ),
                        row=3, col=1
                    )

                fig.update_xaxes(title_text="$k$ (wavenumber)", row=3, col=1)
                fig.update_yaxes(title_text="$|\\hat{f}(k)|$", type='log', row=3, col=1)

            fig.update_xaxes(title_text="$x$", row=1, col=1)
            fig.update_yaxes(title_text="$f(x)$", row=1, col=1)
            fig.update_xaxes(title_text="$x$ (position)", row=2, col=1)
            fig.update_yaxes(title_text="$p$ (momentum)", row=2, col=1)

            # Add annotation explaining the wave front set
            fig.add_annotation(
                x=sing_pos, y=p.max() * 0.8,
                text=f"WF = $\\{{({sing_pos:.1f}, p) : {wf_description}\\}}$",
                showarrow=False,
                bgcolor='rgba(255,255,0,0.8)',
                bordercolor='yellow',
                font=dict(color='black', size=11),
                xref='x2', yref='y2'
            )

            fig.update_layout(
                title=f'Wave Front Set at Singularity $x_0={sing_pos:.1f}$ ({title})<br>Vertical Line in Phase Space: $\\text{{WF}} = \\{{(x_0, p) : {wf_description}\\}}$'
            )

            fig.show()

widget = WaveFrontWidget(title="Wave Front Set Visualization")
widget.display()


## 3. Algebraic Analysis: D-Modules

We now stop differentiating functions and start studying the differentiators themselves.

### 3.1 The Weyl Algebra

Let $D_X$ be the ring of linear differential operators with smooth coefficients. This ring is non-commutative:

$$[\partial_x, x] = 1$$

A system of linear PDEs corresponds to a Left Ideal $I \subset D_X$. The "system" is the quotient module $M = D_X / I$.

Solving the equation $Pu = 0$ corresponds to finding the Hom-set $\text{Hom}_{D_X}(M, \mathcal{O})$, where $\mathcal{O}$ is a function space.

### 3.2 The Characteristic Variety

Just as a matrix has eigenvalues, a D-Module has a **Characteristic Variety** $\text{Char}(M) \subset T^*X$. This is the set of points where the "symbol" (highest order term) of the operator vanishes.

**Bernstein's Inequality:** The dimension of $\text{Char}(M)$ is never small. It is always at least $\dim(X)$ (the dimension of the manifold). Systems where this dimension is exactly $\dim(X)$ are called **Holonomic**. These are the "integrable systems" of the algebraic world (including all Special Functions from Level 1).



In [ ]:
# [Holonomic Variety. A visualization of the Characteristic Variety for the Hypergeometric Equation. It shows the "Lagrangian" surface in phase space that encodes the system's singularities.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class HolonomicVarietyWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['equation_type'] = ParameterSlider(
            name='equation_type',
            value=0.0,
            min_val=0.0,
            max_val=3.0,
            step=1.0,
            description='Equation (0: Hypergeometric, 1: Bessel, 2: Legendre, 3: Airy)'
        )
        self.sliders['a_param'] = ParameterSlider(
            name='a_param',
            value=1.0,
            min_val=0.5,
            max_val=2.0,
            step=0.1,
            description='Parameter $a$'
        )
        self.sliders['b_param'] = ParameterSlider(
            name='b_param',
            value=1.0,
            min_val=0.5,
            max_val=2.0,
            step=0.1,
            description='Parameter $b$'
        )
        self.sliders['show_lagrangian'] = ParameterSlider(
            name='show_lagrangian',
            value=1.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Show Lagrangian (0: Off, 1: On)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            eq_type = int(params['equation_type'])
            a = params['a_param']
            b = params['b_param']
            show_lag = params['show_lagrangian'] > 0.5

            # Different equations have different characteristic varieties
            if eq_type == 0:
                # Hypergeometric: z(1-z) u'' + [c - (a+b+1)z] u' - a b u = 0
                eq_name = 'Hypergeometric'
                z = np.linspace(0.01, 0.99, 150)
                zeta_real = np.linspace(-4, 4, 150)
                Z, ZETA_R = np.meshgrid(z, zeta_real)
                c = 1.0
                char_poly = Z * (1 - Z) * ZETA_R**2 + (c - (a + b + 1) * Z) * ZETA_R - a * b
                sing_points = [(0.0, 0.0), (1.0, 0.0)]
            elif eq_type == 1:
                # Bessel: z² u'' + z u' + (z² - ν²) u = 0, where ν = a
                eq_name = f'Bessel (ν={a:.1f})'
                z = np.linspace(0.1, 3, 150)
                zeta_real = np.linspace(-4, 4, 150)
                Z, ZETA_R = np.meshgrid(z, zeta_real)
                char_poly = Z**2 * ZETA_R**2 + (Z**2 - a**2)
                sing_points = [(0.0, 0.0)]
            elif eq_type == 2:
                # Legendre: (1-z²) u'' - 2z u' + n(n+1) u = 0, where n = a
                eq_name = f'Legendre (n={a:.1f})'
                z = np.linspace(-0.99, 0.99, 150)
                zeta_real = np.linspace(-4, 4, 150)
                Z, ZETA_R = np.meshgrid(z, zeta_real)
                n = a
                char_poly = (1 - Z**2) * ZETA_R**2 - n * (n + 1)
                sing_points = [(-1.0, 0.0), (1.0, 0.0)]
            else:  # eq_type == 3
                # Airy: u'' - z u = 0
                eq_name = 'Airy'
                z = np.linspace(-3, 2, 150)
                zeta_real = np.linspace(-4, 4, 150)
                Z, ZETA_R = np.meshgrid(z, zeta_real)
                char_poly = ZETA_R**2 - Z
                sing_points = []  # No fixed singular points

            # Characteristic variety is where char_poly = 0
            # For holonomic systems, this is a Lagrangian submanifold of dimension dim(X)

            fig = self.plot_manager.create_plotly_figure(width=750, height=500)

            # Plot characteristic variety as contour where char_poly ≈ 0
            # Use symmetric contour levels around zero
            max_val = np.max(np.abs(char_poly))

            fig.add_trace(go.Contour(
                x=z,
                y=zeta_real,
                z=char_poly,
                contours=dict(
                    start=-max_val*0.3,
                    end=max_val*0.3,
                    size=max_val*0.04,
                    showlines=True,
                    coloring='heatmap'
                ),
                colorscale='RdBu',
                showscale=True,
                colorbar=dict(title="Char. Poly.", x=1.02),
                name='Characteristic Polynomial'
            ))

            # Find and plot the Lagrangian branches (where char_poly = 0)
            if show_lag:
                # For each z, find ζ where char_poly is zero
                lagrangian_branches = []
                for i, z_val in enumerate(z):
                    char_slice = char_poly[:, i]
                    # Find zero crossings more precisely
                    zero_crossings = np.where(np.abs(char_slice) < max_val*0.1)[0]
                    if len(zero_crossings) > 0:
                        # Sample points along the zero contour
                        for idx in zero_crossings[::max(1, len(zero_crossings)//20)]:
                            lagrangian_branches.append((z_val, zeta_real[idx]))

                if len(lagrangian_branches) > 0:
                    lag_z, lag_zeta = zip(*lagrangian_branches)
                    fig.add_trace(go.Scatter(
                        x=list(lag_z),
                        y=list(lag_zeta),
                        mode='markers',
                        marker=dict(size=4, color='yellow', symbol='circle', line=dict(width=1, color='black')),
                        name='Lagrangian Points'
                    ))

            # For specific equations, compute explicit branches
            if eq_type == 0:  # Hypergeometric
                # Solve quadratic: z(1-z) ζ² + [c - (a+b+1)z] ζ - a b = 0
                c = 1.0  # Already defined above, but ensure it's set
                disc = (c - (a + b + 1) * Z)**2 + 4 * a * b * Z * (1 - Z)
                disc = np.maximum(disc, 0)
                denom = 2 * Z * (1 - Z) + 1e-10
                zeta1 = (-(c - (a + b + 1) * Z) + np.sqrt(disc)) / denom
                zeta2 = (-(c - (a + b + 1) * Z) - np.sqrt(disc)) / denom

                # Plot branches as lines (more efficient than individual markers)
                # Sample points to avoid too many traces
                step = max(1, len(z) // 50)
                z_sampled = z[::step]
                zeta1_sampled = zeta1[0, ::step]
                zeta2_sampled = zeta2[0, ::step]

                # Filter out invalid values
                valid1 = np.isfinite(zeta1_sampled) & (np.abs(zeta1_sampled) < 10)
                valid2 = np.isfinite(zeta2_sampled) & (np.abs(zeta2_sampled) < 10)

                if np.any(valid1):
                    fig.add_trace(go.Scatter(
                        x=z_sampled[valid1], y=zeta1_sampled[valid1],
                        mode='lines',
                        line=dict(color='blue', width=2),
                        name='Branch 1: $\\zeta_+$'
                    ))
                if np.any(valid2):
                    fig.add_trace(go.Scatter(
                        x=z_sampled[valid2], y=zeta2_sampled[valid2],
                        mode='lines',
                        line=dict(color='red', width=2),
                        name='Branch 2: $\\zeta_-$'
                    ))
            elif eq_type == 1:  # Bessel: z² ζ² + (z² - ν²) = 0 → ζ = ±√(ν²/z² - 1)
                # For z > ν, we have real solutions
                # Characteristic equation: z² ζ² + (z² - ν²) = 0
                # Solving: ζ² = (ν² - z²)/z² = (ν/z)² - 1
                # Real solutions exist when (ν/z)² - 1 >= 0, i.e., z <= ν
                # But for z > ν, we get imaginary solutions
                # For visualization, we'll show where the characteristic polynomial is zero
                z_valid = z[z > 0.1]  # Avoid division by zero
                if len(z_valid) > 0:
                    # Compute the argument of sqrt
                    sqrt_arg = (a**2 / z_valid**2) - 1
                    # Only compute sqrt where argument is non-negative
                    valid_mask = sqrt_arg >= 0
                    z_real = z_valid[valid_mask]

                    if len(z_real) > 0:
                        sqrt_arg_valid = sqrt_arg[valid_mask]
                        zeta_bessel_plus = np.sqrt(sqrt_arg_valid)
                        zeta_bessel_minus = -np.sqrt(sqrt_arg_valid)

                        fig.add_trace(go.Scatter(
                            x=z_real, y=zeta_bessel_plus,
                            mode='lines',
                            line=dict(color='blue', width=2),
                            name='Branch: $\\zeta_+$'
                        ))
                        fig.add_trace(go.Scatter(
                            x=z_real, y=zeta_bessel_minus,
                            mode='lines',
                            line=dict(color='red', width=2),
                            name='Branch: $\\zeta_-$'
                        ))
            elif eq_type == 2:  # Legendre: (1-z²) ζ² - n(n+1) = 0 → ζ = ±√(n(n+1)/(1-z²))
                z_valid = z[np.abs(z) < 0.99]  # Avoid singularities
                if len(z_valid) > 0:
                    n = a
                    denom_leg = 1 - z_valid**2
                    zeta_leg_plus = np.sqrt(n * (n + 1) / denom_leg)
                    zeta_leg_minus = -np.sqrt(n * (n + 1) / denom_leg)
                    fig.add_trace(go.Scatter(
                        x=z_valid, y=zeta_leg_plus,
                        mode='lines',
                        line=dict(color='blue', width=2),
                        name='Branch: $\\zeta_+$'
                    ))
                    fig.add_trace(go.Scatter(
                        x=z_valid, y=zeta_leg_minus,
                        mode='lines',
                        line=dict(color='red', width=2),
                        name='Branch: $\\zeta_-$'
                    ))
            elif eq_type == 3:  # Airy: ζ² = z
                # Explicit solution: ζ = ±√z
                z_positive = z[z >= 0]
                if len(z_positive) > 0:
                    zeta_plus = np.sqrt(z_positive)
                    zeta_minus = -np.sqrt(z_positive)
                    fig.add_trace(go.Scatter(
                        x=z_positive, y=zeta_plus,
                        mode='lines',
                        line=dict(color='blue', width=3),
                        name='Branch: $\\zeta = +\\sqrt{z}$'
                    ))
                    fig.add_trace(go.Scatter(
                        x=z_positive, y=zeta_minus,
                        mode='lines',
                        line=dict(color='red', width=3),
                        name='Branch: $\\zeta = -\\sqrt{z}$'
                    ))

            # Mark singular points
            if sing_points:
                sing_z, sing_zeta = zip(*sing_points)
                fig.add_trace(go.Scatter(
                    x=list(sing_z),
                    y=list(sing_zeta),
                    mode='markers',
                    marker=dict(size=15, color='orange', symbol='x', line=dict(width=2, color='black')),
                    name='Singular Points'
                ))

            fig.update_layout(
                title=f'Characteristic Variety for {eq_name} Equation<br>Lagrangian Surface in Phase Space $(z, \\zeta)$',
                xaxis_title='$z$ (position)',
                yaxis_title='$\\zeta$ (momentum)',
                showlegend=True
            )

            fig.show()

widget = HolonomicVarietyWidget(title="Holonomic Characteristic Variety")
widget.display()


## 4. The Riemann-Hilbert Correspondence

This is the bridge between Analysis (Level 7) and Topology (Level 3).

### 4.1 Regular Singularities

For regular holonomic D-modules (generalizing "Regular Singular Points" from Level 1), the solutions form a locally constant sheaf (a bundle of vector spaces) over the manifold minus the singularities.

### 4.2 The Correspondence

**Kashiwara (1980) / Mebkhout (1980):** There is an equivalence of categories between Regular Holonomic D-Modules and **Perverse Sheaves** (topological objects constructed from the geometry of the singularities).

This means that calculating the cohomology of a differential equation (Analysis) is equivalent to calculating the intersection cohomology of a geometric space (Topology).


## 5. Resurgence & Alien Calculus

We return to the divergent series of Level 1 with the advanced tools of Level 7.

### 5.1 The Borel Plane

Jean Écalle (1981) introduced **Resurgence Theory**. We take the Borel transform of a divergent asymptotic series. The result is an analytic function with singularities in the complex Borel plane.

These singularities are not obstacles; they contain non-perturbative information (instantons).

### 5.2 Alien Derivatives

We define **Alien Derivatives** $\Delta_\omega$ that measure how the Borel transform "jumps" across a singularity at $\omega$. These operators form a Lie algebra, quantifying the ambiguity of resummation and systematically encoding the Stokes Phenomenon.



In [11]:
# [Alien Calculus. A complex plane plot of the Borel Transform of a divergent series. The user can inspect the singularities (poles/cuts). The widget calculates the "Alien Derivative" at a singularity, extracting the non-perturbative coefficient (e.g., the tunneling rate).]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class AlienCalculusWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['series_type'] = ParameterSlider(
            name='series_type',
            value=0.0,
            min_val=0.0,
            max_val=4.0,
            step=1.0,
            description='Series (0: Euler, 1: QED, 2: Anharmonic, 3: Double Well, 4: Custom)'
        )
        self.sliders['sing_point'] = ParameterSlider(
            name='sing_point',
            value=1.0,
            min_val=0.1,
            max_val=3.0,
            step=0.1,
            description='Singularity Point $\\omega$'
        )
        self.sliders['show_stokes'] = ParameterSlider(
            name='show_stokes',
            value=0.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Show Stokes Lines (0: Off, 1: On)'
        )
        self.sliders['order'] = ParameterSlider(
            name='order',
            value=10.0,
            min_val=5.0,
            max_val=20.0,
            step=1.0,
            description='Series Order $N$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            stype = int(params['series_type'])
            omega = params['sing_point']

            # Complex Borel plane ζ
            re = np.linspace(-3, 3, 200)
            im = np.linspace(-3, 3, 200)
            Re, Im = np.meshgrid(re, im)
            Zeta = Re + 1j * Im

            show_stokes = params['show_stokes'] > 0.5
            N = int(params['order'])

            # Borel transform B(ζ) for different series
            if stype == 0:
                # Euler series: sum (-1)^n n! z^n
                # Borel transform: B(ζ) = 1/(1 + ζ)
                B = 1.0 / (1.0 + Zeta + 1e-10)
                title = 'Euler Series'
                sing_positions = [(-1.0, 0.0)]
                stokes_angles = [np.pi]  # Real negative axis
            elif stype == 1:
                # QED perturbation: factorial growth
                B = 1.0 / (1.0 - Zeta + 1e-10) + 0.5 / (2.0 - Zeta + 1e-10)
                title = 'Perturbative QED'
                sing_positions = [(1.0, 0.0), (2.0, 0.0)]
                stokes_angles = [0.0]  # Real positive axis
            elif stype == 2:
                # Anharmonic oscillator: double factorial growth
                # B(ζ) has singularities at positive integers
                B = 1.0 / (1.0 - Zeta + 1e-10) + 0.3 / (2.0 - Zeta + 1e-10) + 0.1 / (3.0 - Zeta + 1e-10)
                title = 'Anharmonic Oscillator'
                sing_positions = [(1.0, 0.0), (2.0, 0.0), (3.0, 0.0)]
                stokes_angles = [0.0]
            elif stype == 3:
                # Double well: instanton singularities
                # B(ζ) has singularities at ω = 1, 2, 3, ... (instanton actions)
                B = 1.0 / (1.0 - Zeta + 1e-10) + 0.2 / (2.0 - Zeta + 1e-10)
                title = 'Double Well (Instantons)'
                sing_positions = [(1.0, 0.0), (2.0, 0.0)]
                stokes_angles = [0.0]
            else:  # stype == 4
                # Custom: multiple singularities with complex structure
                B = 1.0 / (1.0 - Zeta + 1e-10) + 0.5 / (1.0 + 1j - Zeta + 1e-10) + 0.5 / (1.0 - 1j - Zeta + 1e-10)
                title = 'Custom (Complex Sings)'
                sing_positions = [(1.0, 0.0), (1.0, 1.0), (1.0, -1.0)]
                stokes_angles = [0.0, np.pi/4, -np.pi/4]

            # Magnitude for plot
            mag = np.abs(B)
            mag = np.clip(mag, 0, 10)  # Clip for visualization

            # Phase for color coding
            phase = np.angle(B)

                        # Alien derivative at ω: measures the jump/discontinuity across the singularity
            # Δ_ω measures how the Borel transform "jumps" when crossing a Stokes line
            alien_deriv_real = 0.0
            alien_deriv_imag = 0.0
            alien_deriv_mag = 0.0

            # Find closest singularity to omega
            if sing_positions:
                # For real singularities, find closest
                closest_sing = min(sing_positions, key=lambda s: np.sqrt((s[0] - omega)**2 + s[1]**2))
                dist = np.sqrt((closest_sing[0] - omega)**2 + closest_sing[1]**2)

                if dist < 0.5:
                    # Compute approximate alien derivative (related to residue)
                    # For a simple pole at ζ = ω, Δ_ω ≈ 2πi * Res(B, ω)
                    if stype == 0:
                        # Euler: B(ζ) = 1/(1+ζ), Res at -1 is 1
                        alien_deriv_imag = 2.0 * np.pi
                    elif stype in [1, 2, 3]:
                        # Simple poles: Res = 1
                        alien_deriv_imag = 2.0 * np.pi
                    else:
                        # Complex singularities: compute from distance
                        alien_deriv_real = 0.5 * np.pi
                        alien_deriv_imag = 1.5 * np.pi

                    alien_deriv_mag = np.sqrt(alien_deriv_real**2 + alien_deriv_imag**2)

            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=('Borel Transform Magnitude $|B(\\zeta)|$', 'Borel Transform Phase $\\arg(B(\\zeta))$'),
                horizontal_spacing=0.15
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=450,
                width=900,
                showlegend=True
            )

            # Left plot: Magnitude
            fig.add_trace(
                go.Heatmap(
                    x=re, y=im, z=mag,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="$|B(\\zeta)|$", x=0.45)
                ),
                row=1, col=1
            )

            # Mark singularities
            for idx, (s_re, s_im) in enumerate(sing_positions):
                fig.add_trace(
                    go.Scatter(
                        x=[s_re], y=[s_im],
                        mode='markers',
                        marker=dict(size=15, color='red', symbol='x', line=dict(width=2, color='white')),
                        name='Singularity',
                        showlegend=(idx == 0)
                    ),
                    row=1, col=1
                )

            # Mark omega point
            fig.add_trace(
                go.Scatter(
                    x=[omega], y=[0.0],
                    mode='markers',
                    marker=dict(size=12, color='yellow', symbol='circle', line=dict(width=2, color='black')),
                    name=f'$\\omega={omega:.1f}$'
                ),
                row=1, col=1
            )

            # Right plot: Phase
            fig.add_trace(
                go.Heatmap(
                    x=re, y=im, z=phase,
                    colorscale='HSV',
                    showscale=True,
                    colorbar=dict(title="$\\arg(B(\\zeta))$", x=1.02)
                ),
                row=1, col=2
            )

            # Mark singularities in phase plot too
            for s_re, s_im in sing_positions:
                fig.add_trace(
                    go.Scatter(
                        x=[s_re], y=[s_im],
                        mode='markers',
                        marker=dict(size=15, color='red', symbol='x', line=dict(width=2, color='white')),
                        name='Singularity',
                        showlegend=False
                    ),
                    row=1, col=2
                )

            fig.update_xaxes(title_text="$\\text{Re}(\\zeta)$", row=1, col=1)
            fig.update_yaxes(title_text="$\\text{Im}(\\zeta)$", row=1, col=1)
            fig.update_xaxes(title_text="$\\text{Re}(\\zeta)$", row=1, col=2)
            fig.update_yaxes(title_text="$\\text{Im}(\\zeta)$", row=1, col=2)

            # Add annotation for alien derivative
            alien_text = f'$\Delta_{{{omega:.1f}}} \approx {alien_deriv_imag:.2f}i$'
            fig.add_annotation(
                x=omega, y=0.5,
                text=alien_text,
                showarrow=True,
                arrowhead=2,
                arrowcolor='yellow',
                bgcolor='rgba(0,0,0,0.8)',
                bordercolor='yellow',
                font=dict(color='yellow', size=12),
                xref='x1', yref='y1'
            )

            fig.update_layout(
                title=f'Borel Plane for {title} Divergent Series<br>Alien Derivative at $\\omega={omega:.1f}$ (Non-Perturbative Term)'
            )

            fig.show()

widget = AlienCalculusWidget(title="Alien Calculus in Resurgence Theory")
widget.display()


## 6. Quantization & TQFT

Finally, we close the loop by viewing the differential equation as a quantum state.

### 6.1 Geometric Quantization

The phase space $T^*M$ is a symplectic manifold. Quantization is the process of replacing functions on $T^*M$ with operators on a Hilbert space. Differential equations are just the "classical limits" of Quantum Field Theories (QFT).

### 6.2 Topological Quantum Field Theory (TQFT)

In TQFT (Atiyah, 1988), the evolution of the system depends only on the topology of spacetime, not the metric. The "invariants" we discovered—Index of operators, Chern numbers, Euler characteristics—are all observables in a TQFT. The differential equation has dissolved; only the topology remains.


---

### References

* **Atiyah, M.** (1988). *Topological quantum field theories*.

* **Bernstein, I. N.** (1971). *Modules over a ring of differential operators*.

* **Écalle, J.** (1981). *Les fonctions résurgentes*.

* **Hörmander, L.** (1971). *Fourier integral operators. I*.

* **Kashiwara, M.** (1980). *Faisceaux constructibles et systèmes holonomes d'équations aux dérivées partielles*.

* **Mebkhout, Z.** (1980). *Sur le problème de Riemann-Hilbert*.

* **Sato, M.** (1959). *Theory of hyperfunctions*.
